In [1]:
from willitfit.scrapers.IKEA import product_info_and_update_csv_database



In [2]:
article_code = ['s39128564','90499066','s29178074','40258154']
path_to_csv = 'ikea_database.csv'


product_info_and_update_csv_database(article_code,path_to_csv)    

,width,high,length,weight,packeges,subarticle_code,article_code
0,6.0,4.0,49.0,0.28,1.0,902.568.93,s39128564
1,6.0,4.0,49.0,0.28,1.0,402.568.95,s39128564
2,6.0,4.0,74.0,0.43,1.0,302.779.59,s39128564
3,6.0,4.0,74.0,0.43,1.0,802.145.68,s39128564
4,48.0,2.0,59.0,3.35,6.0,202.145.71,s39128564
5,48.0,2.0,59.0,3.35,6.0,102.463.32,s39128564
6,61.0,7.0,245.0,44.16,2.0,902.779.61,s39128564
0,78.0,65.0,96.0,41.20,1.0,904.990.66,90499066
0,58.0,55.0,59.0,6.70,1.0,003.204.93,s29178074
1,58.0,55.0,59.0,6.70,1.0,803.204.94,s29178074


## Draft



In [ ]:
def extract_numeric_product_to_list(y):
    """
    ['Artikelnummer:',
     'Breite: 61 cm',
     'Höhe: 7 cm',
     'Länge: 245 cm',
     
     'Gewicht: 44.16 kg',
     'Paket(e): 2']
    """
    numeric_product_info = []
    for x in [item for subitem in y for item in subitem.split()]:   
        try:
            x = float(x) 
            numeric_product_info.append(x)       
        except (ValueError, TypeError):
            pass
    return numeric_product_info

def packages_dimensions_weights(page):
    """
    [(
    article_code (str),
    item_count (int),
    [(
        package_id (int),
        package_length (int),
        package_width (int),
        package_height (int),
        package_weight (float)
    )])]
    """
    info = page.find_all('div',  {"class": 'range-revamp-product-details__container'})
    number = page.find_all('span',  {"class": 'range-revamp-product-identifier__value'})
    list_info = []
    for i,(x,y) in enumerate(zip(number,info),start = 1):
        y = [info.text for info in y.find_all('span',  {"class": 'range-revamp-product-details__label'})]
        list_info.append([(x.text,i),extract_numeric_product_to_list(y)])  
    return list_info
    
#packages_dimensions_weights(page)

def products_info_to_df(article_code,path_to_csv):
    
    all_ordered_product_df = pd.DataFrame()
    for i,x in enumerate(article_code):
    
        page = scrap_product(x,item_count=None)

        df = packages_dimensions_weights(page)
        df['article_code'] = article_code[i]
        all_ordered_product_df = all_ordered_product_df.append(df)
        
    
    return all_ordered_product_df


def rename_columns(df):
    new_columns_name = ['width','high','length','weight','packeges','subarticle_code', 'article_code']
    rename_columns = {x:y for x,y in zip(df.columns,new_columns_name)}
    df.rename(columns = rename_columns, inplace = True)
    
    return df